In [34]:
import cv2
import torch

image = cv2.imread('/media/quocviet/SSD Data/Viet_FAS/CelebA_Spoof/image_preprocessed/spoof_face/495357.png')
image = cv2.resize(image, (224,224))
image = torch.tensor(image, dtype=torch.long)
image.shape

torch.Size([224, 224, 3])

In [35]:
image = image.unfold(0, 16, 16).unfold(1, 16, 16)
image = image.reshape(-1, 3, 16, 16)
image = image.reshape(image.size(0),-1)
image.shape

torch.Size([196, 768])